## 1. Importar Bibliotecas

In [16]:
import pandas as pd
import json
from collections import Counter
import requests
import csv
import os

In [18]:
# Criar diretórios necessários
os.makedirs("../data/input", exist_ok=True)
os.makedirs("../data/output", exist_ok=True)
print("Diretórios criados/verificados: data/input, data/output")

Diretórios criados/verificados: data/input, data/output


## 2. Coleta de Dados da API

In [19]:
url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjce/_search"
api_key = "APIKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw=="

payload = json.dumps({
    "size": 10000,
    "query": {"match": {"assuntos.codigo": "12487"}},
    "sort": [{"dataAjuizamento": {"order": "desc"}}]
})

headers = {
    'Authorization': api_key,
    'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)
dados_dict = response.json()
print(f"Total de processos encontrados na API: {dados_dict['hits']['total']['value']}")
print(f"Processos retornados nesta consulta: {len(dados_dict['hits']['hits'])}")

# Salvando em csv (números de processos para referência)
processos = dados_dict["hits"]["hits"]

numeros = []
for processo in processos:
    numero_processo = processo["_source"]["numeroProcesso"]
    numeros.append(numero_processo)

Total de processos encontrados na API: 6046
Processos retornados nesta consulta: 6046


## 3. Processamento dos Dados (Python Dataframe)

In [20]:
# Extrair dados dos processos
processos = []

for hit in dados_dict['hits']['hits']:
    processo = hit['_source']
    numero_processo = processo['numeroProcesso']
    grau = processo['grau']
    classe = processo['classe']['nome']
    assuntos = processo['assuntos']
    data_ajuizamento = processo['dataAjuizamento']
    ultima_atualizacao = processo['dataHoraUltimaAtualizacao']
    formato = processo['formato']['nome']
    codigo = processo['orgaoJulgador']['codigo']
    orgao_julgador = processo['orgaoJulgador']['nome']
    municipio = processo['orgaoJulgador']['codigoMunicipioIBGE']
    
    try:
        movimentos = processo['movimentos']
    except:
        movimentos = []
    
    processos.append([numero_processo, classe, data_ajuizamento, ultima_atualizacao, formato,
                     codigo, orgao_julgador, municipio, grau, assuntos, movimentos])

df = pd.DataFrame(processos, columns=['numero_processo', 'classe', 'data_ajuizamento', 'ultima_atualizacao',
                                      'formato', 'codigo', 'orgao_julgador', 'municipio', 'grau', 'assuntos', 'movimentos'])

print(f"Total de processos no DataFrame: {len(df)}")
df.sample(5)

Total de processos no DataFrame: 6046


,numero_processo,classe,data_ajuizamento,ultima_atualizacao,formato,codigo,orgao_julgador,municipio,grau,assuntos,movimentos
284,02066953020228060112,Apelação Cível,20250414000000,2025-07-11T14:03:21.776Z,Eletrônico,50905,GADES - TEREZE NEUMANN DUARTE CHAVES,2304400,G2,"[{'codigo': 12487, 'nome': 'Fornecimento de me...","[{'complementosTabelados': [{'codigo': 2, 'val..."
1552,30047447320228060001,Apelação Cível,20240313000000,2024-09-18T21:10:40.592Z,Eletrônico,50895,GADES - LUIZ EVALDO GONÇALVES LEITE,2304400,G2,"[{'codigo': 12496, 'nome': 'Oncológico'}, {'co...","[{'complementosTabelados': [{'codigo': 18, 'va..."
1508,02452858420238060001,Apelação Cível,20240327000000,2025-05-10T15:02:08.595Z,Eletrônico,50900,GADES - RAIMUNDO NONATO SILVA SANTOS,2304400,G2,"[{'codigo': 12487, 'nome': 'Fornecimento de me...","[{'complementosTabelados': [{'codigo': 19, 'va..."
5904,06318271120188060000,Agravo de Instrumento,20181205000000,2024-08-31T07:17:33.087Z,Eletrônico,50900,GADES - RAIMUNDO NONATO SILVA SANTOS,2304400,G2,"[{'codigo': 12487, 'nome': 'Fornecimento de me...","[{'complementosTabelados': [{'codigo': 19, 'va..."
1029,02011987720248060043,Procedimento do Juizado Especial Cível,20240722000000,2024-10-09T09:28:45.896Z,Eletrônico,82398,2ª VARA CIVEL DA COMARCA DE BARBALHA,2301901,G1,"[{'codigo': 12487, 'nome': 'Fornecimento de me...","[{'complementosTabelados': [{'codigo': 4, 'val..."


## 4. Análise de Decisões
#### Aqui podemos restringir os termos para "procedente/improcedente" 

In [22]:
# Extrair decisões dos movimentos
decisoes_por_processo = []
tipos_decisao_contagem = []

for idx, row in df.iterrows():
    numero = row['numero_processo']
    movimentos = row['movimentos']
    grau = row['grau']
    
    decisoes_encontradas = []

    if movimentos:
        for mov in movimentos:
            nome_mov = mov.get('nome', '')
            
            termos_decisao = [
                "Procedência",
                "Improcedência",
                "Improcedência do pedido e improcedência do pedido contraposto",
                "Procedência do pedido e procedência do pedido contraposto"
            ]
            
            # Identificar decisões
            if (any(palavra in nome_mov for palavra in termos_decisao) and grau == 'G1') :
                decisoes_encontradas.append(nome_mov)
                tipos_decisao_contagem.append(nome_mov)

    if decisoes_encontradas:
        decisoes_por_processo.append({
            'numero_processo': numero,
            'decisoes': decisoes_encontradas
        })

print(f"Processos com decisões: {len(decisoes_por_processo)} de {len(df)}")
print(f"\nTipos de decisões encontradas:")
for tipo, count in Counter(tipos_decisao_contagem).most_common(10):
    print(f"  {tipo}: {count}")

Processos com decisões: 1799 de 6046

Tipos de decisões encontradas:
  Procedência: 1284
  Procedência em Parte: 259
  Improcedência: 222
  Procedência em parte do pedido e improcedência do pedido contraposto: 30
  Improcedência do pedido e improcedência do pedido contraposto: 14
  Procedência do pedido e improcedência do pedido contraposto: 5
  Procedência do Pedido - Reconhecimento pelo réu: 3
  Procedência em parte do pedido e procedência do pedido contraposto: 1
  Procedência do pedido e procedência do pedido contraposto: 1


In [24]:
# Criar DataFrame de decisões
decisoes_lista = []

for item in decisoes_por_processo:
    for decisao in item['decisoes']:
        decisoes_lista.append({
            'numero_processo': item['numero_processo'],
            'tipo_decisao': decisao,
        })

df_decisoes = pd.DataFrame(decisoes_lista)
df_decisoes = df_decisoes.drop_duplicates(subset='numero_processo', keep='first')
df_decisoes.head(10)

,numero_processo,tipo_decisao
0,30039390220258060071,Improcedência
1,02187331420258060001,Procedência
2,30465508320258060001,Procedência em Parte
3,30048329320258060167,Procedência
4,30415587920258060001,Procedência em Parte
5,02162085920258060001,Procedência
6,30355412720258060001,Procedência
7,30022788520258060071,Procedência
8,30303751420258060001,Procedência em Parte
9,30292622520258060001,Procedência


## 5. Seleção de TODOS os Registros

**Nova Estratégia:**
- Coletar **TODOS** os registros com decisão (sem seleção estratificada)
- Incluir todos os tipos de decisão:
  - Procedência
  - Improcedência
  - Improcedência do pedido e improcedência do pedido contraposto
  - Procedência do pedido e procedência do pedido contraposto

**Objetivo:**
- Maximizar volume de dados para ML
- Aplicar filtro automático de processos SEM texto (segredo de justiça) no pipeline
- Esperar ~68.3% dos processos com texto público disponível

In [26]:
# Separar decisões por tipo
df_procedencia = df_decisoes[df_decisoes['tipo_decisao'] == 'Procedência'].copy()
df_improcedencia = df_decisoes[df_decisoes['tipo_decisao'] == 'Improcedência'].copy()
df_improcedencia_contraposto = df_decisoes[df_decisoes['tipo_decisao'] == 'Improcedência do pedido e improcedência do pedido contraposto'].copy()
df_procedencia_contraposto = df_decisoes[df_decisoes['tipo_decisao'] == 'Procedência do pedido e procedência do pedido contraposto'].copy()

# NOVA ESTRATÉGIA: Pegar TODOS os registros (sem seleção estratificada)
df_decisoes_completo = pd.concat([
    df_procedencia,
    df_improcedencia,
    df_improcedencia_contraposto,
    df_procedencia_contraposto
])

print(f"\n=== TODOS OS REGISTROS COM DECISÃO ===")
print(f"Total de registros: {len(df_decisoes_completo)}")
print(f"\nDistribuição por tipo:")
print(f"  - Procedência: {len(df_procedencia)}")
print(f"  - Improcedência: {len(df_improcedencia)}")
print(f"  - Improcedência (contraposto): {len(df_improcedencia_contraposto)}")
print(f"  - Procedência (contraposto): {len(df_procedencia_contraposto)}")

# Estimativa de processos com texto (68.3% disponíveis)
processos_com_texto_estimado = int(len(df_decisoes_completo) * 0.683)
print(f"\nEstimativa de processos COM texto público:")
print(f"  {len(df_decisoes_completo)} × 68.3% = ~{processos_com_texto_estimado} processos")

# Mostrar distribuição
df_decisoes_completo.head(10)


=== TODOS OS REGISTROS COM DECISÃO ===
Total de registros: 1497

Distribuição por tipo:
  - Procedência: 1264
  - Improcedência: 219
  - Improcedência (contraposto): 13
  - Procedência (contraposto): 1

Estimativa de processos COM texto público:
  1497 × 68.3% = ~1022 processos


,numero_processo,tipo_decisao
1,02187331420258060001,Procedência
3,30048329320258060167,Procedência
5,02162085920258060001,Procedência
6,30355412720258060001,Procedência
7,30022788520258060071,Procedência
9,30292622520258060001,Procedência
13,30012724320258060071,Procedência
14,30193154420258060001,Procedência
15,30186884020258060001,Procedência
16,30182752720258060001,Procedência


## 6. Exportar Resultados

In [28]:
# Salvar DataFrames em CSV
cwd = os.getcwd()

df.to_csv('../data/output/processos_completo.csv', sep=',', header=True, index=False)
df_decisoes_completo.to_csv('../data/output/decisoes_resumo.csv', sep=',', header=True, index=False)

# Salvar números de processos completos em CSV
numeros_completos = df_decisoes_completo['numero_processo'].unique()
with open('../data/output/numeros_processos.csv', 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['numeroProcesso'])
    for num in numeros_completos:
        writer.writerow([num])

# Salvar dados completos em JSON (todos os processos com decisão)
processos_completos = [p for p in dados_dict['hits']['hits'] if p['_source']['numeroProcesso'] in numeros_completos]
dados_completos_todos = {
    'hits': {
        'hits': processos_completos,
        'total': {'value': len(processos_completos)}
    }
}

with open('../data/output/dados_completos.json', 'w', encoding='utf-8') as f:
    json.dump(dados_completos_todos, f, indent=2, ensure_ascii=False)

# Salvar processos com decisões
with open('../data/output/processos_com_decisoes.json', 'w', encoding='utf-8') as f:
    json.dump(decisoes_por_processo, f, indent=2, ensure_ascii=False)

print("\n=== ARQUIVOS EXPORTADOS ===")
print(f"  - data/output/processos_completo.csv")
print(f"  - data/output/decisoes_resumo.csv ({len(df_decisoes_completo)} registros)")
print(f"  - data/output/numeros_processos.csv ({len(numeros_completos)} processos únicos)")
print(f"  - data/output/dados_completos.json ({len(processos_completos)} processos)")
print(f"  - data/output/processos_com_decisoes.json")
print(f"\nOBS: Pipeline automaticamente filtrará processos sem texto (segredo de justiça)")


=== ARQUIVOS EXPORTADOS ===
  - data/output/processos_completo.csv
  - data/output/decisoes_resumo.csv (1497 registros)
  - data/output/numeros_processos.csv (1497 processos únicos)
  - data/output/dados_completos.json (1876 processos)
  - data/output/processos_com_decisoes.json

OBS: Pipeline automaticamente filtrará processos sem texto (segredo de justiça)


## 7. Próximos Passos: Coleta Completa de Dados

Após gerar os arquivos básicos acima, execute o pipeline completo de coleta de dados:

### Opção 1: Pipeline Completo (3 etapas)
```bash
python coletar_dados_ml.py
```

Isso executará:
1. **Scraping** - Coleta juiz e requerente do TJCE
2. **Inferência de Sexo** - Infere sexo dos nomes
3. **Geração de Features** - Gera dataset ML com 29 features

### Opção 2: Executar Etapas Individuais
```bash
# Apenas scraping
python coletar_dados_ml.py --etapa scraping

# Apenas inferência de sexo
python coletar_dados_ml.py --etapa inferir_sexo

# Apenas geração de features
python coletar_dados_ml.py --etapa features
```

### Arquivos Gerados
- `data/dados_processos_tjce.csv` - Dados do scraping
- `data/dados_processos_com_sexo.csv` - Dados + sexo inferido
- `data/dataset_ml_completo.csv` - Dataset final (29 features)